In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from tqdm import tqdm
import requests
import time

ModuleNotFoundError: No module named 'spotipy'

In [ ]:
# Spotify API credentials (Must replace)
CLIENT_ID = ''  # Spotify Client ID
CLIENT_SECRET = ''  # Spotify Client Secret

session = requests.Session()
session.timeout = (5, 10)  # (connect timeout, read timeout) in seconds
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET), requests_session=session)

# Check for the API
try:
    results = sp.search(q='artist:Taylor Swift', type='artist', limit=1)
    if results['artists']['items']:
        artist = results['artists']['items'][0]
        print("API is working. Retrieved artist information:")
        print(f"Name: {artist['name']}")
        print(f"Genres: {artist['genres']}")
        print(f"Popularity: {artist['popularity']}")
    else:
        print("API is connected but no artist information was found.")
except Exception as e:
    print(f"API check failed: {e}")

API is working. Retrieved artist information:
Name: Taylor Swift
Genres: ['pop']
Popularity: 100


In [ ]:
# CSV file loading
path = os.getcwd()
data = pd.read_csv(path + r"\spotify_data\Spotify_Dataset_V3.csv", delimiter=";")
data.drop_duplicates(subset = 'Title', inplace = True)
print(data.shape)

(7457, 20)


In [ ]:
genres = []
artist_genre_cache = {}

def get_genres_from_tracks(track_ids):
    """Fetch genres for multiple tracks using a batch request, with caching for artists."""
    batch_genres = []  # Collect genres for this batch only
    try:
        # Track information for multiple track IDs
        response = sp.tracks(track_ids)
        tracks_info = response['tracks']
        
        for track_info in tracks_info:
            if track_info:  # Check for existence
                artist_id = track_info['artists'][0]['id']
                
                # Check cache before making an API call
                if artist_id in artist_genre_cache:
                    batch_genres.append(artist_genre_cache[artist_id])
                else:
                    # Artist info is fetched for the genre
                    artist_info = sp.artist(artist_id)
                    genre = artist_info.get('genres', [])
                    artist_genre_cache[artist_id] = genre
                    batch_genres.append(genre)
            else:
                batch_genres.append(None)
    except spotipy.exceptions.SpotifyException as e:
        if e.http_status == 429:
            handle_rate_limit()
            return get_genres_from_tracks(track_ids)  # Retry after waiting
        else:
            print(f"API error: {e}")
            batch_genres.extend([None] * len(track_ids))
    except Exception as e:
        print(f"Error fetching genres for batch of track IDs {track_ids}: {e}")
        batch_genres.extend([None] * len(track_ids))
    
    genres.extend(batch_genres)  # Only extend `genres` once, after handling each batch

# Rate limit handler
def handle_rate_limit():
    retry_after = 5  # Default wait time if retry-after is not provided
    print(f"Rate limit reached. Retrying after {retry_after} seconds.")
    time.sleep(retry_after)

# Batches, to minimize API calls
batch_size = 20  
track_ids = data['id'].tolist()  

for i in tqdm(range(0, len(track_ids), batch_size), desc="Processing songs"):
    batch = track_ids[i:i + batch_size]
    get_genres_from_tracks(batch)

# Adding the genres to the DataFrame and saving to a new CSV
data['Genre'] = genres[:len(data)]  
data.to_csv('output_with_genres.csv', index=False)

print("Genres fetched and saved to output_with_genres.csv")

Processing songs:   2%|▏         | 6/373 [00:06<05:35,  1.09it/s]HTTP Error for GET to https://api.spotify.com/v1/artists/4bthk9UfsYUYdcFyqxmSUU with Params: {} returned 429 due to API rate limit exceeded


Rate limit reached. Retrying after 5 seconds.


Processing songs:   4%|▍         | 15/373 [00:17<04:55,  1.21it/s]HTTP Error for GET to https://api.spotify.com/v1/artists/2LIk90788K0zvyj2JJVwkJ with Params: {} returned 429 due to API rate limit exceeded


Rate limit reached. Retrying after 5 seconds.


Processing songs:   9%|▉         | 35/373 [00:35<05:09,  1.09it/s]HTTP Error for GET to https://api.spotify.com/v1/artists/6ZjFtWeHP9XN7FeKSUe80S with Params: {} returned 429 due to API rate limit exceeded


...

Rate limit reached. Retrying after 5 seconds.


Processing songs:  91%|█████████ | 338/373 [03:09<00:09,  3.89it/s]HTTP Error for GET to https://api.spotify.com/v1/artists/4kYGAK2zu9EAomwj3hXkXy with Params: {} returned 429 due to API rate limit exceeded


Rate limit reached. Retrying after 5 seconds.


Processing songs: 100%|██████████| 373/373 [03:25<00:00,  1.81it/s]

Genres fetched and saved to output_with_genres.csv
